In [6]:
%pip install ipyleaflet Shapely geopandas fiona


  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/27/27/2687abaa2ac02b5814e2929a5033da1e5d132c19a904dc56f77f63fd6eb9/geopandas-0.14.0-py3-none-any.whl.metadata
  Using cached geopandas-0.14.0-py3-none-any.whl.metadata (1.5 kB)
  Obtaining dependency information for fiona from https://files.pythonhosted.org/packages/7f/27/b24c1610c7ae5716709321f04d38d7b8b71ed531f80df4f697b9ad99cfc3/fiona-1.9.5-cp311-cp311-win_amd64.whl.metadata
  Using cached fiona-1.9.5-cp311-cp311-win_amd64.whl.metadata (51 kB)
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/79/95/eb68113c5b5737c342bde1bab92705dabe69c16299c5a122616e50f1fbd6/pyproj-3.6.1-cp311-cp311-win_amd64.whl.metadata
  Using cached pyproj-3.6.1-cp311-cp311-win_amd64.whl.metadata (31 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
Using cached geopandas-0.14.0-py3-none-any.whl (1.1 

In [1]:
from shapely.geometry import Polygon

def get_area_polygon(bounds):
    """
    Genera un polígono a partir de las coordenadas proporcionadas en el argumento JSON.

    Args:
        bounds (dict): Un diccionario con las coordenadas de los límites del área.

    Returns:
        Polygon: El polígono que representa el área a consultar.
    """

    south = bounds["south"]
    west = bounds["west"]
    north = bounds["north"]
    east = bounds["east"]

    # Define las coordenadas del polígono en sentido horario
    polygon_coordinates = [
        (south, west),
        (north, west),
        (north, east),
        (south, east),
        (south, west),  # Añade el primer punto nuevamente para cerrar el polígono
    ]

    return Polygon(polygon_coordinates)

In [16]:
from pykml import parser
from shapely.geometry.polygon import Polygon
from ipyleaflet import Map, DrawControl, GeoJSON

from pykml import parser
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

"""
Modulo que para carga un archivo KML y encuentra poligonos dentro de un area
"""

from pykml import parser
from shapely.geometry.polygon import Polygon

#loggin
import logging
logging.basicConfig(level=logging.DEBUG, filename='sample.log')
logging.debug('Sample dict log: %s', {'name' : "John", 'age': 10})

def find_polygons_in_area(kml_file, area_polygon):
    print("CONSULTANDO AREA : " + str(area_polygon))
    with open(kml_file, 'rb') as f:
        root = parser.parse(f).getroot()
    polygons_within_area = []
    for folder in root.Document.Folder:
        folder_name = folder.name.text.strip()
        print("ZONA: " + folder_name)
        for placemark in folder.Placemark:
            try:
                coordinates = placemark.Polygon.outerBoundaryIs.LinearRing.coordinates.text.strip()
                coordinates = coordinates.split()  # Separa las coordenadas
                # Solo latitud y longitud
                coordinates = [c.split(',')[:2] for c in coordinates]
                # Crea un poligono con las coordenadas
                polygon = Polygon(coordinates)
                
                if polygon.intersects(area_polygon):
                    poligono = {
                            'folder_name': folder_name,
                            'placemark': placemark.name.text,
                            'poligonos': coordinates
                    }
                    # Si el poligono intersecta con el area especificada, agregalo a la lista
                    logging.debug("Encontre un poligono", poligono)
                    polygons_within_area.append(poligono)
            except AttributeError:
                pass

    return polygons_within_area

# Define una función para manejar el evento on_draw del control de dibujo
def handle_draw(event, action, geo_json):
    # Extrae las coordenadas del polígono dibujado
    if action == 'created':
        coordinates = geo_json['geometry']['coordinates'][0]
        print("COORDENADAS RAW\n")
        print(coordinates)
        # Convierte las coordenadas a un objeto Polygon de Shapely
        drawn_polygon = Polygon(coordinates)
        # Realiza las operaciones que necesites con el polígono dibujado
        # Por ejemplo, puedes imprimir las coordenadas o verificar la intersección con otras geometrías
        logging.debug("Polígono dibujado:", drawn_polygon)
        # Llama a la función para actualizar el mapa
        refresh_map(drawn_polygon)
    pass

# Crea un mapa
m = Map(center=(-12.090819,-77.057704), zoom=16)  # Centra el mapa en San Isidro, Perú

# Agrega el control de dibujo
draw_control = DrawControl(
    circle={}, 
    polyline={}, 
    polygon={'shapeOptions': {'color': 'blue', 'fillOpacity': 0.2}}, 
    rectangle={'shapeOptions': {'color': 'blue', 'fillOpacity': 0.2}}, 
    edit=True, 
    remove=True 
)
m.add_control(draw_control)

# Define una función para actualizar el mapa con las capas GeoJSON
def refresh_map(area_coordinates):
    m.clear_layers()  # Limpia las capas actuales

    # Agrega el área delimitada por el rectángulo como un polígono
    area_polygon = Polygon(area_coordinates)
    geojson_area = GeoJSON(data=area_polygon.__geo_interface__, style={'color': 'blue', 'fillOpacity': 0.2})
    m.add_layer(geojson_area)

    # Llama a la función para encontrar polígonos en el área
    kml_file = 'DATABASE/mixed_geometry.kml' 
    polygons_within_area = find_polygons_in_area(kml_file, area_polygon)

    # Agrega los polígonos encontrados al mapa como capas GeoJSON
    for polygon_data in polygons_within_area:
        polygon = Polygon(polygon_data['poligonos'])
        geojson = GeoJSON(data=polygon.__geo_interface__, style={'color': 'red', 'fillOpacity': 0.2})
        m.add_layer(geojson)

bounds = {
    "south": -12.099629892051706,
    "west": -77.03630851882374,
    "north": -12.09622046087039,
    "east": -77.03201698439992
}

area_detected = get_area_polygon(bounds)

# Convierte el objeto Polygon a GeoJSON
geojson = area_detected.__geo_interface__
print(f"Area Detected {geojson}") 
# Crea una capa GeoJSON en el mapa con el polígono
geojson_layer = GeoJSON(data=geojson, style={'color': 'red', 'fillOpacity': 0.2} )
m.add_layer(geojson_layer)

# Configura el manejo del evento on_draw
draw_control.on_draw(handle_draw)

# Muestra el mapa
m

Area Detected {'type': 'Polygon', 'coordinates': (((-12.099629892051706, -77.03630851882374), (-12.09622046087039, -77.03630851882374), (-12.09622046087039, -77.03201698439992), (-12.099629892051706, -77.03201698439992), (-12.099629892051706, -77.03630851882374)),)}


Map(center=[-12.090819, -77.057704], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…